In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

In [ ]:
Behavior_files, Phot_files, Pupil_files, MetaData_files, MetaDataX_files, ANIMAL_IDs, ANIMAL_VARs = DirectoryAlloc_testedit(ROOTDIR, 200, 0)

loopIDX = [0]  
subjIDX = [ANIMAL_VARs[ANIMAL_IDs[i]] for i in loopIDX]

# Load MetaSPK_test and initialize index variables
MetaSPK_test = loadmat(ROOTDIR + '2021-22_Attention/NP 2023-12/MetaSPK_test')['MetaSPK_test']
OFCIDX = range(len(MetaSPK_test))
len_idx = len(OFCIDX)

pairwise_CCF_FR0_Success = [None] * len_idx
pairwise_CCF_FR0_Punish = [None] * len_idx

idx_st = CrossSampling(tt_success, -3)
idx_ed = CrossSampling(tt_success, -1)

# Calculate pairwise correlations for success and punish data
for i in range(len_idx):
    success_corr, success_pvals = np.corrcoef(Lumped_PriorSuccess_FR0[i][:, idx_st:idx_ed].T)
    punish_corr, punish_pvals = np.corrcoef(Lumped_PriorPunish_FR0[i][:, idx_st:idx_ed].T)
    pairwise_CCF_FR0_Success[i] = (success_corr, success_pvals)
    pairwise_CCF_FR0_Punish[i] = (punish_corr, punish_pvals)

# Display example correlation heatmaps for the session 8
i = 7  
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

sns.heatmap(pairwise_CCF_FR0_Success[i][0], cmap='jet', vmin=-0.8, vmax=1, ax=axes[0], cbar=True)
axes[0].set_title("Success Pairwise Correlation")
sns.heatmap(pairwise_CCF_FR0_Punish[i][0], cmap='jet', vmin=-0.8, vmax=1, ax=axes[1], cbar=True)
axes[1].set_title("Punish Pairwise Correlation")
plt.tight_layout()
plt.show()

# Plot histogram and scatter for significant correlations
plt.figure(figsize=(10, 6))
for i in range(len_idx):
    # Mask p-values above 0.05 and only keep upper triangular values
    success_pvals = np.triu(pairwise_CCF_FR0_Success[i][1], 1)
    success_pvals[np.where(success_pvals == 0) | (success_pvals > 0.05)] = np.nan
    success_pvals[np.where(success_pvals <= 0.05)] = 1
    significant_CCF_Success = pairwise_CCF_FR0_Success[i][0] * success_pvals

    punish_pvals = np.triu(pairwise_CCF_FR0_Punish[i][1], 1)
    punish_pvals[np.where(punish_pvals == 0) | (punish_pvals > 0.05)] = np.nan
    punish_pvals[np.where(punish_pvals <= 0.05)] = 1
    significant_CCF_Punish = pairwise_CCF_FR0_Punish[i][0] * punish_pvals

    plt.scatter(np.nanmean(np.abs(significant_CCF_Punish)), np.nanmean(np.abs(significant_CCF_Success)), s=20, c='k', marker='o')

plt.xlabel('Mean Absolute Correlation (Punish)')
plt.ylabel('Mean Absolute Correlation (Success)')
plt.title('Significant Pairwise Correlations for Success and Punish Conditions')
plt.show()
